<a href="https://colab.research.google.com/github/alina518/practice/blob/main/CIFAR_10_Object_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
#configuring path of json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
#dataset api
!kaggle competitions download -c cifar-10

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 4, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.12/dist-packages/kaggle/__init__.py", line 6, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 434, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/


In [ ]:
!ls

sample_data


In [ ]:
from zipfile import ZipFile
dataset='/content/cifar-10.zip'
with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('Dataset is extracted')

FileNotFoundError: [Errno 2] No such file or directory: '/content/cifar-10.zip'

In [ ]:
!ls

In [ ]:
!pip install py7zr

In [ ]:
import py7zr
archive=py7zr.SevenZipFile('/content/train.7z',mode='r')
archive.extractall()
archive.close()

In [ ]:
!ls

Importing dependencies

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split

In [ ]:
filename=os.listdir('/content/train')

In [ ]:
#processing labels
labels_data=pd.read_csv('/content/trainLabels.csv')

In [ ]:
labels_data.shape

In [ ]:
labels_data.head(10)

In [ ]:
labels_data.tail()

In [ ]:
labels_data['label'].value_counts()

In [ ]:
labels_dict={'frog':0,
'ship':1,
'truck':2,
'deer':	3,
'automobile':4,
'bird':5,
'horse':6,
'cat':7,
'dog':8,
'airplane':9}
labels=[labels_dict[i] for i in labels_data['label']]
#list contains id of the corresponding elements

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
image_path=cv2.imread('/content/train/2.png')
cv2_imshow(image_path)

In [ ]:
labels_data[labels_data['id']==45788]

In [ ]:
id_list=list(labels_data['id'])

Image Processing

In [ ]:
#convert images to numpy array

train_data_folder='/content/train/'

data=[]
for id in id_list:
  image=Image.open(train_data_folder+str(id)+'.png')
  image=np.array(image)
  data.append(image)

In [ ]:
#convert image_list and labels_list to numpy arrays
x=np.array(data)
y=np.array(labels)

Train Test Split

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=2)

In [ ]:
#scaling the data
x_train=x_train/255
x_test=x_test/255

In [ ]:
#building the neural network
import tensorflow as tf
from tensorflow import keras

In [ ]:
num_of_classes=10
#layers of neural network
model=keras.Sequential([
    keras.layers.Flatten(input_shape=(32,32,3)),
    keras.layers.Dense(64,activation='relu'),
    keras.layers.Dense(num_of_classes,activation='softmax')
])

In [ ]:
#compile the neural network
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['acc'])

In [ ]:
#training the neural network
model.fit(x_train,y_train,validation_split=0.1,epochs=10)

ResNet50

In [ ]:
from tensorflow.keras import Sequential,models,layers
from tensorflow.keras.layers import Dense,Flatten,Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras import optimizers

In [ ]:
convolutional_base=ResNet50(weights='imagenet',include_top=False,input_shape=(256,256,3))
convolutional_base.summary()

In [ ]:
model=models.Sequential()
model.add(layers.UpSampling2D((2,2)))
model.add(layers.UpSampling2D((2,2)))
model.add(layers.UpSampling2D((2,2)))
model.add(convolutional_base)
model.add(layers.Flatten())
model.add(layers.BatchNormalization())
model.add(layers.Dense(128,activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.BatchNormalization())
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.BatchNormalization())
model.add(layers.Dense(num_of_classes,activation='softmax'))

In [ ]:
model.compile(optimizer=optimizers.RMSprop(learning_rate=2e-5),loss='sparse_categorical_crossentropy',metrics=['acc'])

In [ ]:
history=model.fit(x_train,y_train,validation_split=0.1,epochs=10)

In [ ]:
h=history
#plot the loss value
plt.plot(h.history['loss'],label='train loss')
plt.plot(h.history['val_loss'],label='validation loss')
plt.legend()
plt.show()
#plot the accuracy value
plt.plot(h.history['acc'],label='train accuracy')
plt.plot(h.history['val_acc'],label='validation accuracy')
plt.legend()
plt.show()